### Criando um NLP Data Loader

Traduzir idiomas não é uma tarefa fácil, especialmente dadas as complexidades, nuances e contextos culturais embutidos neles. Central para o sucesso desse esforço são os dados - grandes corpora de frases bilíngues que servem como base para seus modelos.

No PyTorch, o data loader desempenha um papel indispensável na gestão de grandes volumes de dados. Para tarefas de processamento de linguagem natural (NLP), os dados geralmente possuem comprimentos variáveis devido às diferentes estruturas e tamanhos das frases entre línguas. O data loader organiza de forma eficiente esses dados em batches de sequências de comprimento variável, garantindo que seus modelos sejam treinados em exemplos diversificados a cada iteração. Esse agrupamento é crucial para aproveitar o poder da computação paralela em GPUs, acelerando o processo de treinamento.

Além disso, o data loader ajuda a embaralhar o conjunto de dados, o que é essencial para evitar que os modelos memorizem a sequência dos dados de treinamento, promovendo uma melhor generalização. Isso é especialmente importante em tarefas de NLP, onde os dados podem estar ordenados ou agrupados por tópicos. O embaralhamento garante que o modelo permaneça robusto e não desenvolva vieses com base na ordem dos dados de entrada.

Por fim, no contexto de NLP, etapas de pré-processamento como tokenização, preenchimento (padding) e conversão para valores numéricos são fundamentais. O data loader no PyTorch fornece ganchos que permitem integrar essas etapas de pré-processamento de maneira fluida, assegurando que os dados textuais brutos sejam transformados em um formato adequado para modelos de aprendizado profundo.

#### 1. Instalar bibliotecas

In [1]:
def warn(*args, **kwargs):
    pass
    
import warnings
warnings.warn = warn
warnings.filterwarnings('ignore')

import importlib.util
import subprocess
import sys

def check_and_install(package, pip_name=None):
    if pip_name is None:
        pip_name = package
    spec = importlib.util.find_spec(package)
    if spec is None:
        print(f"{package} não está instalado. Instalando...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", pip_name])
    else:
        print(f"{package} já está instalado.")

In [2]:
#!pip install -Uqq torchtext #0.14.1
#!pip install -Uqq torch #1.13.1
#!pip install -Uqq spacy
#!pip install -Uqq torchdata #0.5.1
#!pip install -Uqq portalocker>=2.0.0
#!python -m spacy download en_core_web_sm -qq
#!python -m spacy download de_core_news_sm -qq
#!python -m spacy download fr_core_news_sm -qq

In [3]:
# Checando e instalando pacotes
check_and_install('torchtext', 'torchtext==0.15.2')
check_and_install('torch', 'torch==2.0.3')
check_and_install('spacy')
check_and_install('torchdata', 'torchdata==0.5.1')
check_and_install('portalocker')

subprocess.check_call([sys.executable, "-m", "spacy", "download", "en_core_web_sm"])
subprocess.check_call([sys.executable, "-m", "spacy", "download", "de_core_news_sm"])
subprocess.check_call([sys.executable, "-m", "spacy", "download", "fr_core_news_sm"])

torchtext já está instalado.
torch já está instalado.
spacy já está instalado.
torchdata já está instalado.
portalocker não está instalado. Instalando...


0

#### 2. Carregar bibliotecas

In [4]:
import torchtext
print(torchtext.__version__)

0.15.2


In [5]:
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torchtext.datasets import multi30k, Multi30k
from typing import Iterable, List
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader
from torchdata.datapipes.iter import IterableWrapper, Mapper
import torchtext

import torch
import torch.nn as nn
import torch.optim as optim

import numpy as np
import random

#### Conjunto de Dados
No PyTorch, um data set é um objeto que representa uma coleção de amostras de dados. Cada amostra de dados normalmente consiste em uma ou mais características de entrada e seus respectivos rótulos de destino.

#### Data Loader
Um data loader no PyTorch é responsável por carregar e organizar os dados de um data set de forma eficiente. Ele simplifica o processo de iteração sobre um data set, embaralhamento e divisão em batches para o treinamento. Em aplicações de NLP, o data loader é utilizado para processar e transformar os dados textuais, e não apenas para carregar o conjunto de dados.

Os data loaders possuem vários parâmetros importantes, incluindo o data set a ser carregado, o tamanho do batch (determinando quantas amostras por batch), shuffle (para definir se os dados serão embaralhados a cada época) e outros. Eles também fornecem uma interface de iterador, facilitando a iteração sobre batches de dados durante o treinamento.

Agora, você pode se perguntar: "O que é um iterador?"

Um iterador é um objeto que pode ser percorrido em um loop. Ele contém elementos que podem ser iterados e geralmente inclui dois métodos, `__iter__()` e `__next__()`. Quando não há mais elementos para iterar, ele gera uma exceção StopIteration.

Iteradores são comumente usados para percorrer grandes conjuntos de dados sem carregar todos os elementos na memória ao mesmo tempo, tornando o processo mais eficiente em termos de memória. No PyTorch, nem todos os data sets são iteradores, mas todos os data loaders são.

No PyTorch, o data loader processa dados em batches, carregando e processando um lote de cada vez na memória de forma eficiente. O tamanho do batch, que você especifica ao criar o data loader, determina quantas amostras são processadas juntas em cada lote. O propósito do data loader é converter os dados de entrada e os rótulos em batches de tensores com a mesma forma, para que possam ser interpretados por modelos de aprendizado profundo.

Por fim, um data loader pode ser usado para tarefas como tokenização, sequenciamento, padronização do tamanho das amostras e transformação dos dados em tensores que seu modelo possa entender.

#### Conjunto de Dados Personalizado e Data Loader no PyTorch
O conjunto de dados abaixo consiste em uma lista de frases aleatórias, e o objetivo é criar batches de frases para processamento posterior, como o treinamento de um modelo de rede neural.

Primeiro definimos um conjunto de dados personalizado chamado CustomDataset. Esse conjunto de dados herda da classe torch.utils.data.Dataset e é inicializado com uma lista de frases. O conjunto de dados inclui dois métodos essenciais:

- `__init__(self, sentences)`: Inicializa o conjunto de dados com uma lista de frases.
- `__getitem__(self, idx)`: Recupera um item (neste caso, uma frase) em um índice específico, idx.

Em seguida, é criado uma instância do conjunto de dados personalizado (custom_dataset) passando a lista de frases. Além disso, podemos especificar um tamanho de batch (batch_size), que determina quantas frases serão agrupadas em cada batch durante o carregamento de dados.

É criado um DataLoader (dataloader) fornecendo o conjunto de dados personalizado e o tamanho de batch para a classe torch.utils.data.DataLoader. Além disso, é definido o shuffle=True, indicando que as frases serão embaralhadas aleatoriamente antes de serem divididas em batches. Esse embaralhamento é especialmente útil para o treinamento de modelos de aprendizado profundo, pois impede que o modelo aprenda padrões com base na ordem dos dados.

Por fim, itera pelo DataLoader para demonstrar como os dados são carregados em batches. Neste código, o tamanho do batch é definido como 2, o que significa que cada batch conterá duas frases. O DataLoader gerencia eficientemente o carregamento de dados em batches, tornando-o adequado para o treinamento de modelos de aprendizado profundo.

Durante a iteração, as frases em cada batch são impressas para ilustrar como o DataLoader agrupa e apresenta os dados. Esse trecho de código fornece um exemplo fundamental de como configurar um conjunto de dados personalizado e um data loader no PyTorch, uma prática comum em fluxos de trabalho de aprendizado profundo.

In [6]:
sentences = [
    "If you want to know what a man's like, take a good look at how he treats his inferiors, not his equals.",
    "Fame's a fickle friend, Harry.",
    "It is our choices, Harry, that show what we truly are, far more than our abilities.",
    "Soon we must all face the choice between what is right and what is easy.",
    "Youth can not know how age thinks and feels. But old men are guilty if they forget what it was to be young.",
    "You are awesome!"
]

In [21]:
# Define a custom dataset
class CustomDataset(Dataset):

    def __init__(self, sentences):
        self.sentences = sentences

    def __len__(self):
        return len(self.sentences)

    def __getitem__(self, idx):
        return self.sentences[idx]

In [22]:
# Create an instance of your custom dataset
custom_dataset = CustomDataset(sentences)

In [27]:
# Define batch size
batch_size = 2

In [28]:
# Create a DataLoader
dataloader = DataLoader(custom_dataset,
                        batch_size=batch_size,
                        shuffle=True)

In [29]:
# Iterate through the DataLoader
for batch in dataloader:
    print(batch)

['Youth can not know how age thinks and feels. But old men are guilty if they forget what it was to be young.', 'Soon we must all face the choice between what is right and what is easy.']
["Fame's a fickle friend, Harry.", 'It is our choices, Harry, that show what we truly are, far more than our abilities.']
['You are awesome!', "If you want to know what a man's like, take a good look at how he treats his inferiors, not his equals."]


Conforme mostrado acima, os dados são organizados em lotes de 2 frases cada. É importante notar que os modelos de aprendizado profundo só podem compreender dados numéricos, e as palavras não têm sentido para eles. Portanto, o próximo passo é converter essas frases em tensores.

#### Criação de Tensores para Conjunto de Dados Personalizado
Neste exemplo de código, é construído de um conjunto de dados personalizado para tarefas de processamento de linguagem natural (NLP) usando PyTorch. O conjunto de dados consiste em uma lista de frases, e o objetivo é pré-processar essas frases, tokenizá-las e convertê-las em tensores de índices de tokens para uso em modelos de NLP. 

As frases e a classe CustomDataset são utilizadas da mesma forma que no trecho de código anterior. As mudanças feitas na classe CustomDataset são as seguintes:

- `__init__`: O construtor recebe uma lista de frases, uma função de tokenização e um vocabulário (vocab) como entrada.
- `__len__`: Este método retorna o número total de amostras no conjunto de dados.
- `__getitem__`: Este método é responsável por processar uma única amostra. Ele tokeniza a frase usando o tokenizador fornecido e, em seguida, converte os tokens em índices de tensor usando o vocabulário.

Podemos definir um tokenizador usando a função get_tokenizer com a opção basic_english. A tokenização é o processo de dividir um texto em tokens ou palavras individuais. Em seguida, é construído um vocabulário a partir das frases.A função build_vocab_from_iterator é utilizada para construir o vocabulário a partir das frases tokenizadas.

É possível criar uma instância do seu conjunto de dados personalizado, passando as frases, o tokenizador e o vocabulário. Finalmente, imprime o comprimento do conjunto de dados personalizado e amostras dele para ilustração.

In [30]:
sentences = [
    "If you want to know what a man's like, take a good look at how he treats his inferiors, not his equals.",
    "Fame's a fickle friend, Harry.",
    "It is our choices, Harry, that show what we truly are, far more than our abilities.",
    "Soon we must all face the choice between what is right and what is easy.",
    "Youth can not know how age thinks and feels. But old men are guilty if they forget what it was to be young.",
    "You are awesome!"
]

In [31]:
# Define a custom data set
class CustomDataset(Dataset):
    def __init__(self, sentences, tokenizer, vocab):
        self.sentences = sentences
        self.tokenizer = tokenizer
        self.vocab = vocab

    def __len__(self):
        return len(self.sentences)

    def __getitem__(self, idx):
        tokens = self.tokenizer(self.sentences[idx])
        # Convert tokens to tensor indices using vocab
        tensor_indices = [self.vocab[token] for token in tokens]
        return torch.tensor(tensor_indices)

In [32]:
# Tokenizer
tokenizer = get_tokenizer("basic_english")

In [33]:
# Build vocabulary
vocab = build_vocab_from_iterator(map(tokenizer, sentences))

In [34]:
# Create an instance of your custom data set
custom_dataset = CustomDataset(sentences, tokenizer, vocab)

In [36]:
print("Custom Dataset Length:", len(custom_dataset))
print("Sample Items:")
for i in range(6):
    sample_item = custom_dataset[i]
    print(f"Item {i + 1}: {sample_item}")
    print("\n")

Custom Dataset Length: 6
Sample Items:
Item 1: tensor([11, 19, 63, 17, 13,  2,  3, 47,  6, 16, 45,  0, 55,  3, 41, 46, 24, 10,
        43, 61,  9, 44,  0, 14,  9, 33,  1])


Item 2: tensor([35,  6, 16,  3, 38, 40,  0,  8,  1])


Item 3: tensor([12,  5, 15, 31,  0,  8,  0, 57, 53,  2, 18, 62,  4,  0, 36, 49, 56, 15,
        21,  1])


Item 4: tensor([54, 18, 50, 23, 34, 58, 30, 27,  2,  5, 52,  7,  2,  5, 32,  1])


Item 5: tensor([66, 29, 14, 13, 10, 22, 60,  7, 37,  1, 28, 51, 48,  4, 42, 11, 59, 39,
         2, 12, 64, 17, 26, 65,  1])


Item 6: tensor([19,  4, 25, 20])




#### Função de Colagem Personalizada (collate function)
Uma função de colagem (collate function) é utilizada no contexto de carregamento e agrupamento de dados em aprendizado de máquina, especialmente quando se lida com dados de comprimento variável, como sequências (e.g., texto, séries temporais, sequências de eventos). Seu principal objetivo é preparar e formatar amostras individuais de dados (exemplos) em batches que possam ser processados de forma eficiente por modelos de aprendizado de máquina.

É definida uma função de colagem personalizada chamada collate_fn. Essa função desempenha um papel crucial ao lidar com sequências de comprimentos variados, como frases em NLP. Seu propósito é preencher (pad) as sequências dentro de um batch para que tenham comprimentos iguais, o que é uma etapa comum de pré-processamento em tarefas de NLP.

- `pad_sequence`: Esta função faz parte do PyTorch e é usada para preencher as sequências em um batch, garantindo um comprimento uniforme. Ela recebe um batch de sequências como entrada e as preenche para igualar o comprimento da sequência mais longa. O argumento `padding_value=0` especifica o valor a ser utilizado para o preenchimento.

In [37]:
# Create a custom collate function
def collate_fn(batch):
    # Pad sequences within the batch to have equal lengths
    padded_batch = pad_sequence(batch,
                                batch_first=True,
                                padding_value=0)
    return padded_batch

Na célula acima, ao preencher as sequências, você define batch_first=True. Quando batch_first=True, a saída estará no formato `[batch_size x seq_len]`; caso contrário, estará no formato `[seq_len x batch_size]`. Alguns modelos aceitam a entrada no formato `[batch_size x seq_len]`, enquanto outros precisam que a entrada esteja no formato `[seq_len x batch_size]`. Esse parâmetro cuida de organizar a entrada na forma desejada.

In [38]:
# Create a data loader with the custom collate function with batch_first=True,
dataloader = DataLoader(custom_dataset, 
                        batch_size=batch_size, 
                        collate_fn=collate_fn)

In [39]:
# Iterate through the data loader
for batch in dataloader:
    for row in batch:
        for idx in row:
            words = [vocab.get_itos()[idx] for idx in row]
        print(words)

['if', 'you', 'want', 'to', 'know', 'what', 'a', 'man', "'", 's', 'like', ',', 'take', 'a', 'good', 'look', 'at', 'how', 'he', 'treats', 'his', 'inferiors', ',', 'not', 'his', 'equals', '.']
['fame', "'", 's', 'a', 'fickle', 'friend', ',', 'harry', '.', ',', ',', ',', ',', ',', ',', ',', ',', ',', ',', ',', ',', ',', ',', ',', ',', ',', ',']
['it', 'is', 'our', 'choices', ',', 'harry', ',', 'that', 'show', 'what', 'we', 'truly', 'are', ',', 'far', 'more', 'than', 'our', 'abilities', '.']
['soon', 'we', 'must', 'all', 'face', 'the', 'choice', 'between', 'what', 'is', 'right', 'and', 'what', 'is', 'easy', '.', ',', ',', ',', ',']
['youth', 'can', 'not', 'know', 'how', 'age', 'thinks', 'and', 'feels', '.', 'but', 'old', 'men', 'are', 'guilty', 'if', 'they', 'forget', 'what', 'it', 'was', 'to', 'be', 'young', '.']
['you', 'are', 'awesome', '!', ',', ',', ',', ',', ',', ',', ',', ',', ',', ',', ',', ',', ',', ',', ',', ',', ',', ',', ',', ',', ',']


Agora, você pode tentar `batch_first=False`, que é o valor PADRÃO

In [40]:
# Create a custom collate function
def collate_fn_bfFALSE(batch):
    # Pad sequences within the batch to have equal lengths
    padded_batch = pad_sequence(batch, padding_value=0)
    return padded_batch

In [42]:
# Create a data loader with the custom collate function with batch_first=True,
dataloader_bfFALSE = DataLoader(custom_dataset, 
                                batch_size=batch_size, 
                                collate_fn=collate_fn_bfFALSE)

In [43]:
# Iterate through the data loader
for seq in dataloader_bfFALSE:
    for row in seq:
        #print(row)
        words = [vocab.get_itos()[idx] for idx in row]
        print(words)

['if', 'fame']
['you', "'"]
['want', 's']
['to', 'a']
['know', 'fickle']
['what', 'friend']
['a', ',']
['man', 'harry']
["'", '.']
['s', ',']
['like', ',']
[',', ',']
['take', ',']
['a', ',']
['good', ',']
['look', ',']
['at', ',']
['how', ',']
['he', ',']
['treats', ',']
['his', ',']
['inferiors', ',']
[',', ',']
['not', ',']
['his', ',']
['equals', ',']
['.', ',']
['it', 'soon']
['is', 'we']
['our', 'must']
['choices', 'all']
[',', 'face']
['harry', 'the']
[',', 'choice']
['that', 'between']
['show', 'what']
['what', 'is']
['we', 'right']
['truly', 'and']
['are', 'what']
[',', 'is']
['far', 'easy']
['more', '.']
['than', ',']
['our', ',']
['abilities', ',']
['.', ',']
['youth', 'you']
['can', 'are']
['not', 'awesome']
['know', '!']
['how', ',']
['age', ',']
['thinks', ',']
['and', ',']
['feels', ',']
['.', ',']
['but', ',']
['old', ',']
['men', ',']
['are', ',']
['guilty', ',']
['if', ',']
['they', ',']
['forget', ',']
['what', ',']
['it', ',']
['was', ',']
['to', ',']
['be', ',']
['

Pode ser visto que a primeira dimensão agora é a sequência em vez do lote, o que significa que as sentenças serão quebradas para que cada linha inclua um token de cada sequência. Por exemplo, a primeira linha, (['if', 'fame']), inclui os primeiros tokens de todas as sequências naquele lote. Precisamos estar ciente desse padrão para evitar qualquer confusão ao trabalhar com redes neurais recorrentes (RNNs) e transformadores.

In [48]:
# Iterate through the data loader with batch_first = TRUE
for batch in dataloader:    
    print(batch)
    print("Length of sequences in the batch:",batch.shape[1], "\n---")

tensor([[11, 19, 63, 17, 13,  2,  3, 47,  6, 16, 45,  0, 55,  3, 41, 46, 24, 10,
         43, 61,  9, 44,  0, 14,  9, 33,  1],
        [35,  6, 16,  3, 38, 40,  0,  8,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0]])
Length of sequences in the batch: 27 
---
tensor([[12,  5, 15, 31,  0,  8,  0, 57, 53,  2, 18, 62,  4,  0, 36, 49, 56, 15,
         21,  1],
        [54, 18, 50, 23, 34, 58, 30, 27,  2,  5, 52,  7,  2,  5, 32,  1,  0,  0,
          0,  0]])
Length of sequences in the batch: 20 
---
tensor([[66, 29, 14, 13, 10, 22, 60,  7, 37,  1, 28, 51, 48,  4, 42, 11, 59, 39,
          2, 12, 64, 17, 26, 65,  1],
        [19,  4, 25, 20,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0]])
Length of sequences in the batch: 25 
---


Também temos a opção de utilizar a função collate para tarefas como tokenização, conversão de índices tokenizados e transformação do resultado em um tensor. É importante observar que o conjunto de dados original permanece intocado por essas transformações.

In [49]:
# Define a custom data set
class CustomDataset(Dataset):
    def __init__(self, sentences):
        self.sentences = sentences

    def __len__(self):
        return len(self.sentences)

    def __getitem__(self, idx):
        return self.sentences[idx]

In [50]:
custom_dataset=CustomDataset(sentences)

In [51]:
custom_dataset[0]

"If you want to know what a man's like, take a good look at how he treats his inferiors, not his equals."

In [52]:
def collate_fn(batch):
    # Tokenize each sample in the batch using the specified tokenizer
    tensor_batch = []
    for sample in batch:
        tokens = tokenizer(sample)
        # Convert tokens to vocabulary indices and create a tensor for each sample
        tensor_batch.append(torch.tensor([vocab[token] for token in tokens]))

    # Pad sequences within the batch to have equal lengths using pad_sequence
    # batch_first=True ensures that the tensors have shape (batch_size, max_sequence_length)
    padded_batch = pad_sequence(tensor_batch, batch_first=True)
    
    # Return the padded batch
    return padded_batch

In [53]:
# Create a data loader for the custom dataset
dataloader = DataLoader(
    dataset=custom_dataset,   # Custom PyTorch Dataset containing your data
    batch_size=batch_size,     # Number of samples in each mini-batch
    shuffle=True,              # Shuffle the data at the beginning of each epoch
    collate_fn=collate_fn      # Custom collate function for processing batches
)

In [55]:
for batch in dataloader:
    print(batch)
    print("shape of sample",len(batch), "\n---")

tensor([[54, 18, 50, 23, 34, 58, 30, 27,  2,  5, 52,  7,  2,  5, 32,  1,  0,  0,
          0,  0,  0,  0,  0,  0,  0],
        [66, 29, 14, 13, 10, 22, 60,  7, 37,  1, 28, 51, 48,  4, 42, 11, 59, 39,
          2, 12, 64, 17, 26, 65,  1]])
shape of sample 2 
---
tensor([[12,  5, 15, 31,  0,  8,  0, 57, 53,  2, 18, 62,  4,  0, 36, 49, 56, 15,
         21,  1,  0,  0,  0,  0,  0,  0,  0],
        [11, 19, 63, 17, 13,  2,  3, 47,  6, 16, 45,  0, 55,  3, 41, 46, 24, 10,
         43, 61,  9, 44,  0, 14,  9, 33,  1]])
shape of sample 2 
---
tensor([[35,  6, 16,  3, 38, 40,  0,  8,  1],
        [19,  4, 25, 20,  0,  0,  0,  0,  0]])
shape of sample 2 
---


____
Esse material tem como referência o curso [Generative AI and LLMs: Architecture and Data Preparation](https://www.coursera.org/learn/generative-ai-llm-architecture-data-preparation?specialization=generative-ai-engineering-with-llms)